In [1]:
import pandas as pd


In [2]:
issues_data_silver = pd.read_parquet('../../data/silver/silver_jira_issues.parquet')

In [3]:
finished_issues = issues_data_silver[issues_data_silver.status != "Open"]
valid_issues = finished_issues[finished_issues.dates_quality == 'VALID']
valid_issues.info()

<class 'pandas.core.frame.DataFrame'>
Index: 804 entries, 1 to 989
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   issue_id              804 non-null    object             
 1   issue_type            804 non-null    object             
 2   status                804 non-null    object             
 3   priority              804 non-null    object             
 4   project_id            804 non-null    object             
 5   assignee_id           804 non-null    object             
 6   assignee_name         804 non-null    object             
 7   assignee_email        804 non-null    object             
 8   created_at            804 non-null    datetime64[ns, UTC]
 9   resolved_at           804 non-null    datetime64[ns, UTC]
 10  raw_created_at        804 non-null    object             
 11  raw_resolved_at       804 non-null    object             
 12  is_created_at

In [4]:
import pandas as pd
import numpy as np
import holidays
from datetime import timedelta


def business_hours_diff(start, end, country_holidays):
    """
    Calcula diferença em horas úteis entre dois timestamps.
    Considera dias úteis (seg-sex) e exclui feriados nacionais.
    """
    if pd.isna(start) or pd.isna(end) or start >= end:
        return 0.0

    # Normaliza datas
    start_date = start.normalize()
    end_date = end.normalize()

    # Gera range de dias úteis
    business_days = pd.date_range(
        start=start_date,
        end=end_date,
        freq="B"
    )

    # Remove feriados
    business_days = [
        day for day in business_days
        if day.date() not in country_holidays
    ]

    total_hours = 0.0

    for day in business_days:
        day_start = day
        day_end = day + timedelta(days=1)

        effective_start = max(start, day_start)
        effective_end = min(end, day_end)

        hours = (effective_end - effective_start).total_seconds() / 3600

        if hours > 0:
            total_hours += hours

    return total_hours


In [6]:
br_holidays = holidays.Brazil()

valid_issues["horas_uteis"] = valid_issues.apply(lambda row: business_hours_diff(row["created_at"], row["resolved_at"], br_holidays), axis=1)

C:\Users\FZ7B\AppData\Local\Temp\ipykernel_14924\2725281299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_issues["horas_uteis"] = valid_issues.apply(lambda row: business_hours_diff(row["created_at"], row["resolved_at"], br_holidays), axis=1)


In [9]:
hours_expected = pd.DataFrame({'priority':['High', 'Medium', 'Low'], 'expected_sla':[24, 72, 120]})

valid_issues = valid_issues.merge(hours_expected, on='priority')

In [13]:
valid_issues['is_resolved'] = (valid_issues.status == 'Resolved')
valid_issues

,issue_id,issue_type,status,priority,project_id,assignee_id,assignee_name,assignee_email,created_at,resolved_at,raw_created_at,raw_resolved_at,is_created_at_valid,is_resolved_at_valid,dates_quality,horas_uteis,expected_sla,is_resolved
0,JIRA-0002,Bug,Resolved,High,DT-ENG,u004,Francinne,francinne@fasttrack.com,2025-03-09 12:39:26+00:00,2025-03-10 10:39:26+00:00,2025-03-09T12:39:26Z,2025-03-10T10:39:26Z,True,True,VALID,10.657222,24,True
1,JIRA-0004,Story,Done,High,DT-ENG,u004,Francinne,francinne@fasttrack.com,2025-11-22 20:24:58+00:00,2025-11-23 07:24:58+00:00,2025-11-22T20:24:58Z,2025-11-23T07:24:58Z,True,True,VALID,0.000000,24,False
2,JIRA-0005,Task,Resolved,Medium,DT-ENG,u004,Francinne,francinne@fasttrack.com,2025-01-06 01:08:44+00:00,2025-01-07 09:08:44+00:00,2025-01-06T01:08:44Z,2025-01-07T09:08:44Z,True,True,VALID,32.000000,72,True
3,JIRA-0008,Story,Resolved,High,DT-ENG,u001,Matheus Malta,matheus.malta@fasttrack.com,2025-04-07 22:55:41+00:00,2025-04-08 14:55:41+00:00,2025-04-07T22:55:41Z,2025-04-08T14:55:41Z,True,True,VALID,16.000000,24,True
4,JIRA-0009,Story,Resolved,Medium,DT-ENG,u002,Gionni Lucio,gionni.lucio@fasttrack.com,2025-10-08 21:21:55+00:00,2025-10-12 10:21:55+00:00,2025-10-08T21:21:55Z,2025-10-12T10:21:55Z,True,True,VALID,50.634722,72,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,JIRA-0983,Bug,Resolved,Medium,DT-ENG,u001,Matheus Malta,matheus.malta@fasttrack.com,2025-12-04 08:01:14+00:00,2025-12-07 01:01:14+00:00,2025-12-04T08:01:14Z,2025-12-07T01:01:14Z,True,True,VALID,39.979444,72,True
800,JIRA-0984,Task,Resolved,Low,DT-ENG,u001,Matheus Malta,matheus.malta@fasttrack.com,2025-01-09 21:09:08+00:00,2025-01-14 22:09:08+00:00,2025-01-09T21:09:08Z,2025-01-14T22:09:08Z,True,True,VALID,73.000000,120,True
801,JIRA-0986,Story,Done,Low,DT-ENG,u004,Francinne,francinne@fasttrack.com,2025-02-09 05:45:27+00:00,2025-02-14 09:45:27+00:00,2025-02-09T05:45:27Z,2025-02-14T09:45:27Z,True,True,VALID,105.757500,120,False
802,JIRA-0987,Task,Done,Low,DT-ENG,u003,Guilherme Francisco,guilherme@fasttrack.com,2025-06-22 11:58:34+00:00,2025-06-28 01:58:34+00:00,2025-06-22T11:58:34Z,2025-06-28T01:58:34Z,True,True,VALID,120.000000,120,False
